<a href="https://colab.research.google.com/github/vitorcosta26/IPM-Sergipe/blob/main/IPM_Sergipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# config

In [1]:
# Configuração do drive com a base de dados
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.0 MB/s eta 0:00:00


In [3]:
import xlsxwriter
import pandas as pd

In [4]:
divisao_territorial_brasileira = pd.read_excel('/content/drive/MyDrive/2023/01/Material Produzido - TCC 2023.1/PauloVitor/Dados/tcc_dataset/DTB_2022/RELATORIO_DTB_BRASIL_MUNICIPIO.xlsx', skiprows=6)

# Índices fuzzy de pobreza multidimensional

## fórmulas

In [5]:
def extrair_valores_minimos_maximos(df):
    valores_minimos_maximos = {}
    for coluna in df.iloc[:, 1:].columns:
        valores_minimos_maximos[coluna] = (df[coluna].max(), df[coluna].min())

    return valores_minimos_maximos

def calcular_xij(df, valores_minimos_maximos, colunas_positivas, colunas_negativas):
    resultado = df.copy()

    for i in range(df.shape[0]):
        for j, coluna in enumerate(df.columns[1:], start=1):
            maximo, minimo = valores_minimos_maximos[coluna]

            if coluna in colunas_negativas:
                x_ij = (maximo - df.iloc[i, j]) / (maximo - minimo)
            else:
                x_ij = (df.iloc[i, j] - minimo) / (maximo - minimo)

            resultado.iloc[i, j] = x_ij

    return resultado

def calcular_peso(df, j):
    n = len(df)
    soma_xj = df.iloc[:, j].sum()
    denominador = n - soma_xj
    if denominador == 0:
        return 0
    else:
        return math.log(n / denominador)

def calcular_indice_fuzzy(df, positivas, negativas):
    resultado = df.copy()

    valores_minimos_maximos = extrair_valores_minimos_maximos(df)
    df_xij = calcular_xij(df, valores_minimos_maximos, positivas, negativas)

    pesos = [calcular_peso(df_xij, j) for j in range(1, df_xij.shape[1])]

    numeradores = (df_xij.iloc[:, 1:] * pesos).sum(axis=1)

    denominador = sum(pesos)

    indices_fuzzy = numeradores / denominador

    resultado['indice_fuzzy'] = indices_fuzzy

    return resultado[['nome_municipio', 'indice_fuzzy']]

## 2000

In [6]:
import math
import pandas as pd
microdados_domicilio_2000 = pd.read_csv('/content/drive/MyDrive/2023/01/Material Produzido - TCC 2023.1/PauloVitor/Dados/tcc_dataset/Censo Demográfico/2000/microdados_domicilio_2000.csv')
microdados_pessoa_2000 = pd.read_csv('/content/drive/MyDrive/2023/01/Material Produzido - TCC 2023.1/PauloVitor/Dados/tcc_dataset/Censo Demográfico/2000/microdados_pessoa_2000.csv')

In [7]:
populacao_por_municipio_2000 = microdados_pessoa_2000['id_municipio'].value_counts()
populacao_por_municipio_2000_df = populacao_por_municipio_2000.to_frame().reset_index()
populacao_por_municipio_2000_df.columns = ['id_municipio', 'populacao']

In [8]:
# Crie uma lista vazia para armazenar as informações dos municípios
municipios_pes_2000 = []

# Agrupar os dados por município
grupo_pes_2000 = microdados_pessoa_2000.groupby('id_municipio')

# Loop pelos grupos
for cod_mun, df in grupo_pes_2000:
    populacao_municipio = populacao_por_municipio_2000_df[populacao_por_municipio_2000_df['id_municipio'] == cod_mun]['populacao'].values[0]
    #print(f'{cod_mun} - {populacao_municipio}')
    taxa_alfabetizacao_5_mais = len(df[(df['v4752'] >= 5) & (df['v0428'] == 1)]) / populacao_municipio
    percentual_5_14_nao_alfabetizadas = len(df[((df['v4752'] >= 5) & (df['v4752'] <= 14)) & (df['v0428'] == 2)]) / populacao_municipio
    percentual_15_mais_nao_alfabetizadas = len(df[(df['v4752'] >= 15) & (df['v0428'] == 2)]) / populacao_municipio
    percentual_domicilio_responsável_nao_alfabetizado = len(df[(df['v0402'] == 1) & (df['v0428'] == 2)]) / populacao_municipio
    percentual_10_mais_sem_instrucao_fundamental_incompleto = len(df[(df['v4752'] >= 10) & (df['v4355'] == 1)]) / populacao_municipio
    percentual_10_mais_fundamental_completo_medio_incompleto = len(df[(df['v4752'] >= 10) & (df['v4355'] == 2)]) / populacao_municipio
    percentual_10_mais_medio_completo_superior_incompleto = len(df[(df['v4752'] >= 10) & (df['v0432'] == 7)]) / populacao_municipio
    percentual_10_17_com_filhos = len(df[(df['v4752'] >= 10) & (df['v4752'] <= 17) & (df['v4690'] >= 1)]) / populacao_municipio

    # Adiciona as informações do município à lista
    municipios_pes_2000.append({
        'id_municipio': cod_mun,
        'Taxa de alfabetização de pessoas de 5 anos ou mais 2000': taxa_alfabetizacao_5_mais,
        'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2000': percentual_5_14_nao_alfabetizadas,
        'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2000': percentual_15_mais_nao_alfabetizadas,
        'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2000': percentual_domicilio_responsável_nao_alfabetizado,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2000': percentual_10_mais_sem_instrucao_fundamental_incompleto,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2000': percentual_10_mais_fundamental_completo_medio_incompleto,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2000': percentual_10_mais_medio_completo_superior_incompleto,
        'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2000': percentual_10_17_com_filhos,
    })

# Crie um novo DataFrame a partir da lista de informações dos municípios
df_municipios_pes_2000 = pd.DataFrame(municipios_pes_2000)

# Crie uma lista vazia para armazenar as informações dos municípios
municipios_dom_2000 = []

# Agrupar os dados por município
grupo_dom_2000 = microdados_domicilio_2000.groupby('id_municipio')

# Loop pelos grupos
for cod_mun, df in grupo_dom_2000:
    populacao_municipio = populacao_por_municipio_2000_df[populacao_por_municipio_2000_df['id_municipio'] == cod_mun]['populacao'].values[0]
    #print(f'{cod_mun} - {populacao_municipio}')
    percentual_energia_eletrica = len(df[df['v0213'] != 3]) / populacao_municipio
    percentual_rede_agua = len(df[df['v0207'] != 3]) / populacao_municipio
    percentual_coleta_lixo = len(df[df['v0212'].isin([1, 2])]) / populacao_municipio
    percentual_banheiro = len(df[df['v0209'] >= 1]) / populacao_municipio
    percentual_densidade_por_dormitorio_acima_2_moradores = len(df[df['v7203'] > 2]) / populacao_municipio
    percentual_geladeira = len(df[df['v0215'] == 1]) / populacao_municipio
    percentual_esgotamento_sanitario = len(df[(df.v0208 == 1) | (df.v0208 == 2)]) / populacao_municipio
    valor_medio_mensal_domicilios = df['v7616'].mean()
    percentual_domicilios_rendimento_ate_meio_salario_minimo = (len(df[df['v7617'] <= 0.5]) / populacao_municipio) * 100

    # Adiciona as informações do município à lista
    municipios_dom_2000.append({
        'id_municipio': cod_mun,
        'Percentual de domicílios com acesso a energia elétrica 2000': percentual_energia_eletrica,
        'Percentual de domicílios com acesso a rede de água 2000': percentual_rede_agua,
        'Percentual de domicílios com acesso a coleta de lixo 2000': percentual_coleta_lixo,
        'Percentual de domicílios com banheiro 2000': percentual_banheiro,
        'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2000': percentual_densidade_por_dormitorio_acima_2_moradores,
        'Percentual de domicílios com acesso ao bem durável geladeira 2000': percentual_geladeira,
        'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2000': percentual_esgotamento_sanitario,
        'Valor do rendimento nominal médio mensal dos domicílios 2000': valor_medio_mensal_domicilios,
        'Percentual de domicílios com rendimento de até meio salário mínimo 2000': percentual_domicilios_rendimento_ate_meio_salario_minimo
    })

# Crie um novo DataFrame a partir da lista de informações dos municípios
df_municipios_dom_2000 = pd.DataFrame(municipios_dom_2000)

In [9]:
df_municipios_2000 = pd.merge(df_municipios_pes_2000, df_municipios_dom_2000, left_on='id_municipio', right_on='id_municipio')

In [10]:
df_municipios_2000 = pd.merge(df_municipios_2000, divisao_territorial_brasileira[['Código Município Completo', 'Nome_Município']], left_on='id_municipio', right_on='Código Município Completo')
df_municipios_2000.drop(columns=['id_municipio', 'Código Município Completo'], inplace=True)
df_municipios_2000.rename(columns={'Nome_Município': 'nome_municipio'}, inplace=True)
# Obter a lista de todas as colunas
colunas = df_municipios_2000.columns.tolist()
# Reordenar as colunas movendo "nome_municipio" para a primeira posição
colunas = ['nome_municipio'] + [coluna for coluna in colunas if coluna != 'nome_municipio']
# Reatribuir as colunas ao DataFrame
df_municipios_2000 = df_municipios_2000[colunas]

In [11]:
df_municipios_2000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 18 columns):
 #   Column                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                         --------------  -----  
 0   nome_municipio                                                                                                                 75 non-null     object 
 1   Taxa de alfabetização de pessoas de 5 anos ou mais 2000                                                                        75 non-null     float64
 2   Percentual de pessoas de 5 a 14 anos não alfabetizadas 2000                                                                    75 non-null     float64
 3   Percentual de pessoas de 15 anos ou mais não alfabetizadas 2000                                                 

## 2010

In [12]:
import math
import pandas as pd
microdados_domicilio_2010 = pd.read_csv('/content/drive/MyDrive/2023/01/Material Produzido - TCC 2023.1/PauloVitor/Dados/tcc_dataset/Censo Demográfico/2010/microdados_domicilio_2010.csv')
microdados_pessoa_2010 = pd.read_csv('/content/drive/MyDrive/2023/01/Material Produzido - TCC 2023.1/PauloVitor/Dados/tcc_dataset/Censo Demográfico/2010/microdados_pessoa_2010.csv')

In [13]:
populacao_por_municipio_2010 = microdados_pessoa_2010['id_municipio'].value_counts()
populacao_por_municipio_2010_df = populacao_por_municipio_2010.to_frame().reset_index()
populacao_por_municipio_2010_df.columns = ['id_municipio', 'populacao']

In [14]:
# Crie uma lista vazia para armazenar as informações dos municípios
municipios_pes_2010 = []

# Agrupar os dados por município
grupo_pes_2010 = microdados_pessoa_2010.groupby('id_municipio')

# Loop pelos grupos
for cod_mun, df in grupo_pes_2010:
    populacao_municipio = populacao_por_municipio_2010_df[populacao_por_municipio_2010_df['id_municipio'] == cod_mun]['populacao'].values[0]
    taxa_alfabetizacao_5_mais = len(df[(df['v6036'] >= 5) & (df['v0627'] == 1)]) / populacao_municipio
    percentual_5_14_nao_alfabetizadas = len(df[((df['v6036'] >= 5) & (df['v6036'] <= 14)) & (df['v0627'] == 2)]) / populacao_municipio
    percentual_15_mais_nao_alfabetizadas = len(df[(df['v6036'] >= 15) & (df['v0627'] == 2)]) / populacao_municipio
    percentual_domicilio_responsável_nao_alfabetizado = len(df[(df['v0502'] == 1) & (df['v0627'] == 2)]) / populacao_municipio
    percentual_10_mais_sem_instrucao_fundamental_incompleto = len(df[(df['v6036'] >= 10) & (df['v6400'] == 1)]) / populacao_municipio
    percentual_10_mais_fundamental_completo_medio_incompleto = len(df[(df['v6036'] >= 10) & (df['v6400'] == 2)]) / populacao_municipio
    percentual_10_mais_medio_completo_superior_incompleto = len(df[(df['v6036'] >= 10) & (df['v6400'] == 3)]) / populacao_municipio
    percentual_10_17_com_filhos = len(df[(df['v6036'] >= 10) & (df['v6036'] <= 17) & (df['v0663'] == 1)]) / populacao_municipio

    # Adiciona as informações do município à lista
    municipios_pes_2010.append({
        'id_municipio': cod_mun,
        'Taxa de alfabetização de pessoas de 5 anos ou mais 2010': taxa_alfabetizacao_5_mais,
        'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2010': percentual_5_14_nao_alfabetizadas,
        'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2010': percentual_15_mais_nao_alfabetizadas,
        'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2010': percentual_domicilio_responsável_nao_alfabetizado,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2010': percentual_10_mais_sem_instrucao_fundamental_incompleto,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2010': percentual_10_mais_fundamental_completo_medio_incompleto,
        'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2010': percentual_10_mais_medio_completo_superior_incompleto,
        'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2010': percentual_10_17_com_filhos,
    })

# Crie um novo DataFrame a partir da lista de informações dos municípios
df_municipios_pes_2010 = pd.DataFrame(municipios_pes_2010)

# Crie uma lista vazia para armazenar as informações dos municípios
municipios_dom_2010 = []

# Agrupar os dados por município
grupo_dom_2010 = microdados_domicilio_2010.groupby('id_municipio')

# Loop pelos grupos
for cod_mun, df in grupo_dom_2010:
    populacao_municipio = populacao_por_municipio_2010_df[populacao_por_municipio_2010_df['id_municipio'] == cod_mun]['populacao'].values[0]
    percentual_energia_eletrica = len(df[df['v0211'] != 3]) / populacao_municipio
    percentual_rede_agua = len(df[df["v0209"] != 3]) / populacao_municipio
    percentual_coleta_lixo = len(df[df['v0210'].isin([1, 2])]) / populacao_municipio
    percentual_banheiro = len(df[df['v0205'] == 1]) / populacao_municipio
    percentual_densidade_por_dormitorio_acima_2_moradores = len(df[df['v6203'] > 2]) / populacao_municipio
    percentual_geladeira = len(df[df['v0216'] == 1]) / populacao_municipio
    percentual_esgotamento_sanitario = len(df[(df.v0207 == 1) | (df.v0207 == 2)]) / populacao_municipio
    valor_medio_mensal_domicilios = df['v6529'].mean()
    percentual_domicilios_rendimento_ate_meio_salario_minimo = (len(df[df['v6530'] <= 0.5]) / populacao_municipio) * 100

    # Adiciona as informações do município à lista
    municipios_dom_2010.append({
        'id_municipio': cod_mun,
        'Percentual de domicílios com acesso a energia elétrica 2010': percentual_energia_eletrica,
        'Percentual de domicílios com acesso a rede de água 2010': percentual_rede_agua,
        'Percentual de domicílios com acesso a coleta de lixo 2010': percentual_coleta_lixo,
        'Percentual de domicílios com banheiro 2010': percentual_banheiro,
        'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2010': percentual_densidade_por_dormitorio_acima_2_moradores,
        'Percentual de domicílios com acesso ao bem durável geladeira 2010': percentual_geladeira,
        'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2010': percentual_esgotamento_sanitario,
        'Valor do rendimento nominal médio mensal dos domicílios 2010': valor_medio_mensal_domicilios,
        'Percentual de domicílios com rendimento de até meio salário mínimo 2010': percentual_domicilios_rendimento_ate_meio_salario_minimo
    })

# Crie um novo DataFrame a partir da lista de informações dos municípios
df_municipios_dom_2010 = pd.DataFrame(municipios_dom_2010)

In [15]:
df_municipios_2010 = pd.merge(df_municipios_pes_2010, df_municipios_dom_2010, left_on='id_municipio', right_on='id_municipio')

In [16]:
df_municipios_2010 = pd.merge(df_municipios_2010, divisao_territorial_brasileira[['Código Município Completo', 'Nome_Município']], left_on='id_municipio', right_on='Código Município Completo')
df_municipios_2010.drop(columns=['id_municipio', 'Código Município Completo'], inplace=True)
df_municipios_2010.rename(columns={'Nome_Município': 'nome_municipio'}, inplace=True)
# Obter a lista de todas as colunas
colunas = df_municipios_2010.columns.tolist()
# Reordenar as colunas movendo "nome_municipio" para a primeira posição
colunas = ['nome_municipio'] + [coluna for coluna in colunas if coluna != 'nome_municipio']
# Reatribuir as colunas ao DataFrame
df_municipios_2010 = df_municipios_2010[colunas]

## Ajustes pela inflação e pela taxa de crescimento populacional

[Histórico das metas para a inflação](https://www.bcb.gov.br/controleinflacao/historicometas)

In [17]:
taxa_crescimento_por_municipio = {}

for id_municipio in populacao_por_municipio_2000_df['id_municipio']:
    if id_municipio in populacao_por_municipio_2010_df['id_municipio'].values:
        populacao_2000 = populacao_por_municipio_2000_df.loc[populacao_por_municipio_2000_df['id_municipio'] == id_municipio, 'populacao'].values[0]
        populacao_2010 = populacao_por_municipio_2010_df.loc[populacao_por_municipio_2010_df['id_municipio'] == id_municipio, 'populacao'].values[0]

        taxa_crescimento = ((populacao_2010 - populacao_2000) / populacao_2000) * 100

        taxa_crescimento_por_municipio[id_municipio] = taxa_crescimento

data = [{'id_municipio': key, 'taxa_crescimento': value} for key, value in taxa_crescimento_por_municipio.items()]

df_taxa_crescimento = pd.DataFrame(data)

df_taxa_crescimento = pd.merge(df_taxa_crescimento, divisao_territorial_brasileira[['Código Município Completo', 'Nome_Município']], left_on='id_municipio', right_on='Código Município Completo')
df_taxa_crescimento.drop(columns=['id_municipio', 'Código Município Completo'], inplace=True)
df_taxa_crescimento.rename(columns={'Nome_Município': 'nome_municipio'}, inplace=True)
colunas = df_taxa_crescimento.columns.tolist()
colunas = ['nome_municipio'] + [coluna for coluna in colunas if coluna != 'nome_municipio']
df_taxa_crescimento = df_taxa_crescimento[colunas]

In [18]:
taxa_inflacao = ((5.91 - 5.97) / 5.97) * 100

In [19]:
df_municipios_2000['Valor do rendimento nominal médio mensal dos domicílios 2000'] *= (1 + taxa_inflacao/100)

In [20]:
df_municipios_2000 = df_municipios_2000.merge(df_taxa_crescimento, on='nome_municipio')
colunas_ajuste = df_municipios_2000.columns.difference(['nome_municipio', 'Valor do rendimento nominal médio mensal dos domicílios 2000'])
df_municipios_2000[colunas_ajuste] *= (1 + df_taxa_crescimento['taxa_crescimento'].values[:, None]/100)
df_municipios_2000 = df_municipios_2000.drop('taxa_crescimento', axis=1)

## Índice 2000

In [21]:
# Colunas relação positiva
positivas_2000 = [
    'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2000',
    'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2000',
    'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2000',
    'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2000',
    'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2000',
    'Percentual de domicílios com rendimento de até meio salário mínimo 2000'
]

# Colunas relação negativa
negativas_2000 = [
    'Taxa de alfabetização de pessoas de 5 anos ou mais 2000',
    'Percentual de domicílios com acesso a energia elétrica 2000',
    'Percentual de domicílios com acesso a rede de água 2000',
    'Percentual de domicílios com acesso a coleta de lixo 2000',
    'Percentual de domicílios com banheiro 2000',
    'Percentual de domicílios com acesso ao bem durável geladeira 2000',
    'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2000',
    'Valor do rendimento nominal médio mensal dos domicílios 2000',
]

In [22]:
# Dimensões
renda_2000 = df_municipios_2000[[
    'nome_municipio',
    'Valor do rendimento nominal médio mensal dos domicílios 2000',
    'Percentual de domicílios com rendimento de até meio salário mínimo 2000'
    ]]

saude_2000 = df_municipios_2000[[
    'nome_municipio',
    'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2000',
    'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2000'
    ]]

condicoes_habitacionais_2000 = df_municipios_2000[[
    'nome_municipio',
    'Percentual de domicílios com acesso a energia elétrica 2000',
    'Percentual de domicílios com acesso a rede de água 2000',
    'Percentual de domicílios com acesso a coleta de lixo 2000',
    'Percentual de domicílios com banheiro 2000',
    'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2000',
    'Percentual de domicílios com acesso ao bem durável geladeira 2000',
    ]]

educacao_2000 = df_municipios_2000[[
    'nome_municipio',
    'Taxa de alfabetização de pessoas de 5 anos ou mais 2000',
    'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2000',
    'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2000',
    'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2000',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2000'
    ]]

In [23]:
valores_minimos_maximos_2000 = extrair_valores_minimos_maximos(df_municipios_2000)
x_ij = calcular_xij(df_municipios_2000, valores_minimos_maximos_2000, positivas_2000, negativas_2000)

In [24]:
pesos_2000 = {}
for j, coluna in enumerate(x_ij.columns[1:], start=1):
    peso = calcular_peso(x_ij, j)
    pesos_2000[coluna] = peso

pesos_2000_df = pd.DataFrame.from_dict(pesos_2000, orient='index', columns=['peso'])
pesos_2000_df = pesos_2000_df.reset_index()
pesos_2000_df.columns = ['indicador', 'peso']

In [25]:
indice_fuzzy_renda_2000 = calcular_indice_fuzzy(renda_2000, positivas_2000, negativas_2000)
indice_fuzzy_saude_2000 = calcular_indice_fuzzy(saude_2000, positivas_2000, negativas_2000)
indice_fuzzy_condicoes_habitacionais_2000 = calcular_indice_fuzzy(condicoes_habitacionais_2000, positivas_2000, negativas_2000)
indice_fuzzy_educacao_2000 = calcular_indice_fuzzy(educacao_2000, positivas_2000, negativas_2000)
indice_fuzzy_2000 = calcular_indice_fuzzy(df_municipios_2000, positivas_2000, negativas_2000)

In [26]:
indice_fuzzy_renda_2000.rename(columns={'indice_fuzzy': 'renda_2000'}, inplace=True)
indice_fuzzy_saude_2000.rename(columns={'indice_fuzzy': 'saude_2000'}, inplace=True)
indice_fuzzy_condicoes_habitacionais_2000.rename(columns={'indice_fuzzy': 'condicoes_habitacionais_2000'}, inplace=True)
indice_fuzzy_educacao_2000.rename(columns={'indice_fuzzy': 'educacao_2000'}, inplace=True)
indice_fuzzy_2000.rename(columns={'indice_fuzzy': 'indice_2000'}, inplace=True)

In [27]:
indice_2000 = pd.merge(indice_fuzzy_renda_2000, indice_fuzzy_saude_2000, on='nome_municipio')
indice_2000 = pd.merge(indice_2000, indice_fuzzy_condicoes_habitacionais_2000, on='nome_municipio')
indice_2000 = pd.merge(indice_2000, indice_fuzzy_educacao_2000, on='nome_municipio')
indice_2000 = pd.merge(indice_2000, indice_fuzzy_2000, on='nome_municipio')

In [28]:
import xlsxwriter

writer = pd.ExcelWriter('indice_2000.xlsx', engine='xlsxwriter')

indice_2000.to_excel(writer, index=False, header=True)

writer.save()

<ipython-input-28-f7b6a462928a>:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [29]:
import xlsxwriter

writer = pd.ExcelWriter('indice_fuzzy_2000.xlsx', engine='xlsxwriter')

indice_fuzzy_renda_2000.to_excel(writer, sheet_name='renda_2000', index=False, header=True)
indice_fuzzy_saude_2000.to_excel(writer, sheet_name='saude_2000', index=False, header=True)
indice_fuzzy_condicoes_habitacionais_2000.to_excel(writer, sheet_name='condicoes_habitacionais_2000', index=False, header=True)
indice_fuzzy_educacao_2000.to_excel(writer, sheet_name='educacao_2000', index=False, header=True)
indice_fuzzy_2000.to_excel(writer, sheet_name='indice_fuzzy_2000', index=False, header=True)
pesos_2000_df.to_excel(writer, sheet_name='pesos_2000', index=False, header=True)

writer.save()

<ipython-input-29-ff849d53c34a>:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [30]:
#indice_2000

In [31]:
#indice_fuzzy_renda_2000

In [32]:
#indice_fuzzy_saude_2000

In [33]:
#indice_fuzzy_condicoes_habitacionais_2000

In [34]:
#indice_fuzzy_educacao_2000

In [35]:
#indice_fuzzy_2000

## Índice 2010

In [36]:
# Colunas relação positiva
positivas_2010 = [
    'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2010',
    'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2010',
    'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2010',
    'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2010',
    'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2010',
    'Percentual de domicílios com rendimento de até meio salário mínimo 2010'
    'Mortalidade infantil (até 1 ano de idade) 2010'
]

# Colunas relação negativa
negativas_2010 = [
    'Taxa de alfabetização de pessoas de 5 anos ou mais 2010',
    'Percentual de domicílios com acesso a energia elétrica 2010',
    'Percentual de domicílios com acesso a rede de água 2010',
    'Percentual de domicílios com acesso a coleta de lixo 2010',
    'Percentual de domicílios com banheiro 2010',
    'Percentual de domicílios com acesso ao bem durável geladeira 2010',
    'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2010',
    'Valor do rendimento nominal médio mensal dos domicílios 2010',
]

In [37]:
# Dimensões
renda_2010 = df_municipios_2010[[
    'nome_municipio',
    'Valor do rendimento nominal médio mensal dos domicílios 2010',
    'Percentual de domicílios com rendimento de até meio salário mínimo 2010'
    ]]

saude_2010 = df_municipios_2010[[
    'nome_municipio',
    'Percentual de crianças e adolescentes, de 10 a 17 anos, com filhos 2010',
    'Percentual de domicílios com acesso aos tipos de esgotamento sanitário: rede geral de esgoto ou pluvial ou fossa séptica 2010'
    ]]

condicoes_habitacionais_2010 = df_municipios_2010[[
    'nome_municipio',
    'Percentual de domicílios com acesso a energia elétrica 2010',
    'Percentual de domicílios com acesso a rede de água 2010',
    'Percentual de domicílios com acesso a coleta de lixo 2010',
    'Percentual de domicílios com banheiro 2010',
    'Percentual de domicílios com densidade de moradores por dormitório acima de dois moradores 2010',
    'Percentual de domicílios com acesso ao bem durável geladeira 2010',
    ]]

educacao_2010 = df_municipios_2010[[
    'nome_municipio',
    'Taxa de alfabetização de pessoas de 5 anos ou mais 2010',
    'Percentual de pessoas de 5 a 14 anos não alfabetizadas 2010',
    'Percentual de pessoas de 15 anos ou mais não alfabetizadas 2010',
    'Percentual de domicílios em que a pessoa responsável não é alfabetizada 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: sem instrução e ensino fundamental incompleto 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: fundamental completo e ensino médio incompleto 2010',
    'Percentual de pessoas de 10 anos ou mais segundo o nível de instrução: ensino médio completo e superior incompleto 2010'
    ]]

In [38]:
valores_minimos_maximos_2010 = extrair_valores_minimos_maximos(df_municipios_2010)
x_ij = calcular_xij(df_municipios_2010, valores_minimos_maximos_2010, positivas_2010, negativas_2010)

In [39]:
pesos_2010 = {}
for j, coluna in enumerate(x_ij.columns[1:], start=1):
    peso = calcular_peso(x_ij, j)
    pesos_2010[coluna] = peso

pesos_2010_df = pd.DataFrame.from_dict(pesos_2010, orient='index', columns=['peso'])
pesos_2010_df = pesos_2010_df.reset_index()
pesos_2010_df.columns = ['indicador', 'peso']

In [40]:
indice_fuzzy_renda_2010 = calcular_indice_fuzzy(renda_2010, positivas_2010, negativas_2010)
indice_fuzzy_saude_2010 = calcular_indice_fuzzy(saude_2010, positivas_2010, negativas_2010)
indice_fuzzy_condicoes_habitacionais_2010 = calcular_indice_fuzzy(condicoes_habitacionais_2010, positivas_2010, negativas_2010)
indice_fuzzy_educacao_2010 = calcular_indice_fuzzy(educacao_2010, positivas_2010, negativas_2010)
indice_fuzzy_2010 = calcular_indice_fuzzy(df_municipios_2010, positivas_2010, negativas_2010)

In [41]:
indice_fuzzy_renda_2010.rename(columns={'indice_fuzzy': 'renda_2010'}, inplace=True)
indice_fuzzy_saude_2010.rename(columns={'indice_fuzzy': 'saude_2010'}, inplace=True)
indice_fuzzy_condicoes_habitacionais_2010.rename(columns={'indice_fuzzy': 'condicoes_habitacionais_2010'}, inplace=True)
indice_fuzzy_educacao_2010.rename(columns={'indice_fuzzy': 'educacao_2010'}, inplace=True)
indice_fuzzy_2010.rename(columns={'indice_fuzzy': 'indice_2010'}, inplace=True)

In [42]:
indice_2010 = pd.merge(indice_fuzzy_renda_2010, indice_fuzzy_saude_2010, on='nome_municipio')
indice_2010 = pd.merge(indice_2010, indice_fuzzy_condicoes_habitacionais_2010, on='nome_municipio')
indice_2010 = pd.merge(indice_2010, indice_fuzzy_educacao_2010, on='nome_municipio')
indice_2010 = pd.merge(indice_2010, indice_fuzzy_2010, on='nome_municipio')

In [43]:
import xlsxwriter

writer = pd.ExcelWriter('indice_2010.xlsx', engine='xlsxwriter')

indice_2010.to_excel(writer, index=False, header=True)

writer.save()

<ipython-input-43-f040075ad345>:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [44]:
# Crie um objeto ExcelWriter e especifique o arquivo de saída
writer = pd.ExcelWriter('indice_fuzzy_2010.xlsx', engine='xlsxwriter')

# Salve cada DataFrame em uma planilha diferente
indice_fuzzy_renda_2010.to_excel(writer, sheet_name='renda_2010', index=False, header=True)
indice_fuzzy_saude_2010.to_excel(writer, sheet_name='saude_2010', index=False, header=True)
indice_fuzzy_condicoes_habitacionais_2010.to_excel(writer, sheet_name='condicoes_habitacionais_2010', index=False, header=True)
indice_fuzzy_educacao_2010.to_excel(writer, sheet_name='educacao_2010', index=False, header=True)
indice_fuzzy_2010.to_excel(writer, sheet_name='indice_fuzzy_2010', index=False, header=True)
pesos_2010_df.to_excel(writer, sheet_name='pesos_2010', index=False, header=True)

# Salve e feche o arquivo
writer.save()

<ipython-input-44-12c6b2bf5d17>:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [45]:
#indice_fuzzy_renda_2010

In [46]:
#indice_fuzzy_saude_2010

In [47]:
#indice_fuzzy_condicoes_habitacionais_2010

In [48]:
#indice_fuzzy_educacao_2010

In [49]:
#indice_fuzzy_2010

# Índice final

In [50]:
indice = pd.merge(indice_2000, indice_2010, on='nome_municipio')

In [51]:
import xlsxwriter

writer = pd.ExcelWriter('indice.xlsx', engine='xlsxwriter')

indice.to_excel(writer, index=False, header=True)

writer.save()

<ipython-input-51-165a3a66abc9>:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
